Perform per class feature extraction.

In [4]:
import cv2
import mediapipe as mp
import numpy as np
import os
import re
import tensorflow as tf
from tensorflow.keras.layers import Conv1D, Input, Dense, GlobalAveragePooling1D
from tensorflow.keras.models import Model
import pickle

# Paths to video dataset
VIDEO_PATH = "../model_input/augmented_train_clips"  # Update this
OUTPUT_DIR = "../model_input/output_features"  # Directory to save per-class feature files
ERROR_LOG_FILE = "../model_input/output_features/error_log.txt"

# Initialize Mediapipe models 
mp_face_mesh = mp.solutions.face_mesh
mp_hands = mp.solutions.hands

# Mediapipe Constants
FACE_FEATURES = [33, 263, 61, 291, 199, 159, 145, 386, 374, 152]  # Eyebrows, eyes, and mouth
MAX_HANDS = 2

# Ensure output directory exists
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Function to extract labels from filenames
def extract_label(filename):
    match = re.search(r'([^_]+)_\d+\..*$', filename)
    if match:
        return match.group(1)
    else:
        return "unknown"

# Function to process a single video and extract features
def extract_features_from_video(video_path):
    cap = cv2.VideoCapture(video_path)

    face_features = []
    hand_features = []

    with mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1) as face_mesh, \
         mp_hands.Hands(static_image_mode=False, max_num_hands=2) as hands:

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            # Convert frame to RGB
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            # Face landmark detection
            face_result = face_mesh.process(frame_rgb)
            frame_face_features = []

            if face_result.multi_face_landmarks:
                for face_landmarks in face_result.multi_face_landmarks:
                    for idx in FACE_FEATURES:
                        landmark = face_landmarks.landmark[idx]
                        frame_face_features.append([landmark.x, landmark.y, landmark.z])
            else:
                frame_face_features = [[0, 0, 0]] * len(FACE_FEATURES)

            face_features.append(frame_face_features)

            # Hand landmark detection
            hand_result = hands.process(frame_rgb)
            frame_hand_features = []

            if hand_result.multi_hand_landmarks:
                for hand_landmarks in hand_result.multi_hand_landmarks:
                    for idx in range(21):  # 21 hand landmarks
                        landmark = hand_landmarks.landmark[idx]
                        frame_hand_features.append([landmark.x, landmark.y, landmark.z])
            else:
                frame_hand_features = [[0, 0, 0]] * (21 * MAX_HANDS)

            if len(frame_hand_features) < (21 * MAX_HANDS):
                pad_length = (21 * MAX_HANDS) - len(frame_hand_features)
                frame_hand_features.extend([[0, 0, 0]] * pad_length)

            hand_features.append(frame_hand_features)

        cap.release()

    face_features = np.array(face_features, dtype=np.float32)
    hand_features = np.array(hand_features, dtype=np.float32)

    min_len = min(len(face_features), len(hand_features))
    face_features = face_features[:min_len]
    hand_features = hand_features[:min_len]

    combined_features = np.hstack([
        face_features.reshape(min_len, -1),
        hand_features.reshape(min_len, -1)
    ])

    return combined_features

# Function to save features per class
def save_features_by_class(features, label):
    class_filename = os.path.join(OUTPUT_DIR, f"features_{label}.pkl")
    if os.path.exists(class_filename):
        # Append to existing file
        with open(class_filename, 'rb') as f:
            data = pickle.load(f)
        existing_features = data['features']
        existing_features.append(features)
        data['features'] = existing_features
    else:
        # Create new file
        data = {'features': [features], 'label': label}

    # Save updated data
    with open(class_filename, 'wb') as f:
        pickle.dump(data, f)

# Main processing
if __name__ == "__main__":
    for file in os.listdir(VIDEO_PATH):
        if file.endswith(('.MP4', '.MOV')):
            print(f"Processing {file}...")
            video_file_path = os.path.join(VIDEO_PATH, file)
            label = extract_label(file)
            
            try:
                features = extract_features_from_video(video_file_path)
                save_features_by_class(features, label)
            
            except Exception as e:
                with open(ERROR_LOG_FILE, "a") as log_file:
                    log_file.write(f"Error processing file {file}: {str(e)}\n")
                print(f"Error processing {file}, logged to {ERROR_LOG_FILE}.")
    
    print("Feature extraction complete. Features saved per class.")

ImportError: DLL load failed while importing _framework_bindings: A dynamic link library (DLL) initialization routine failed.